In [2]:
import pandas as pd
import numpy as np
import openpyxl
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [ ]:
df = pd.read_excel('doc_comment_summary.xlsx', header=None)
df.columns = ['Комментарий', 'Тональность']
print(df)

In [ ]:
print(df.describe())

In [ ]:
print(df.info())

In [ ]:
df.dropna(inplace=True)

In [ ]:
print(df.info())

In [ ]:
df = df[(df['Тональность'] != 0) & (df['Тональность'].apply(type) == int)]
df['Положительная тональность'] = (df['Тональность'] > 0).astype(bool)
print(df)

In [ ]:
value_counts = df['Положительная тональность'].value_counts()
true_count = value_counts[True]
false_count = value_counts[False]
ratio = true_count / false_count
print(ratio)

In [ ]:
train_data, test_data, train_labels, test_labels = train_test_split(
    df['Комментарий'], df['Положительная тональность'], test_size=0.9, random_state=2023)
train_data = train_data.astype(str)
test_data = test_data.astype(str)
train_labels = train_labels.astype(str)
test_labels = test_labels.astype(str)
print(test_labels)

In [ ]:
pipeline = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', LogisticRegression())
])

# Обучение модели на всей тренировочной выборке
pipeline.fit(train_data, train_labels)

# Применение модели к тестовой выборке
predicted_labels = pipeline.predict(test_data)

# Вычисление значения метрики правильности
accuracy = accuracy_score(test_labels, predicted_labels)
print("Точность:", accuracy)

In [ ]:
pipeline = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', DecisionTreeClassifier())
])

# Обучение модели на всей тренировочной выборке
pipeline.fit(train_data, train_labels)

# Применение модели к тестовой выборке
predicted_labels = pipeline.predict(test_data)

# Вычисление значения метрики правильности
accuracy = accuracy_score(test_labels, predicted_labels)
print("Точность:", accuracy)

In [ ]:
param_grid = {
    'tfidf__ngram_range': [(1, 1), (1, 2), (2, 2)],
    'tfidf__max_df': [0.5, 0.75, 1.0],
    'tfidf__min_df': [1, 2, 3]
}

# задаем токенизатор-векторизатор и модель
tfidf = TfidfVectorizer(stop_words='english')
lr = LogisticRegression()

# задаем пайплайн для перебора параметров с помощью GridSearchCV
pipeline = Pipeline([
    ('tfidf', tfidf),
    ('lr', lr)
])

# запускаем поиск лучших параметров с помощью кросс-валидации на тренировочной выборке
grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(train_data, train_labels)

# выводим результаты
print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

# вычисляем качество модели на отложенной выборке с лучшими параметрами
best_model = grid_search.best_estimator_
test_pred = best_model.predict(test_data)
test_acc = accuracy_score(test_labels, test_pred)
print("Test accuracy:", test_acc)

In [ ]:
param_grid_rf = {
    'tfidf__ngram_range': [(1, 1), (1, 2), (2, 2)],
    'tfidf__max_df': [0.5, 0.75, 1.0],
    'tfidf__min_df': [1, 2, 3],
    'rf__n_estimators': [50, 100, 200],
    'rf__max_depth': [10, 20, 30, None]
}

# задаем сетку параметров для TfidfVectorizer и SVC
param_grid_svm = {
    'tfidf__ngram_range': [(1, 1), (1, 2), (2, 2)],
    'tfidf__max_df': [0.5, 0.75, 1.0],
    'tfidf__min_df': [1, 2, 3],
    'svm__C': [0.1, 1, 10, 100],
    'svm__gamma': ['scale', 'auto']
}

# задаем токенизатор-векторизатор и модели
tfidf = TfidfVectorizer(stop_words='english')
rf = RandomForestClassifier()
svm = SVC()

# задаем пайплайн для перебора параметров с помощью GridSearchCV
pipeline_rf = Pipeline([
    ('tfidf', tfidf),
    ('rf', rf)
])

pipeline_svm = Pipeline([
    ('tfidf', tfidf),
    ('svm', svm)
])

# запускаем поиск лучших параметров с помощью кросс-валидации на тренировочной выборке
grid_search_rf = GridSearchCV(pipeline_rf, param_grid=param_grid_rf, cv=5, n_jobs=-1, verbose=1)
grid_search_rf.fit(train_data, train_labels)

grid_search_svm = GridSearchCV(pipeline_svm, param_grid=param_grid_svm, cv=5, n_jobs=-1, verbose=1)
grid_search_svm.fit(train_data, train_labels)

# выводим результаты
print("Best parameters for RandomForestClassifier:", grid_search_rf.best_params_)
print("Best score for RandomForestClassifier:", grid_search_rf.best_score_)

print("Best parameters for SVM:", grid_search_svm.best_params_)
print("Best score for SVM:", grid_search_svm.best_score_)

# вычисляем качество моделей на отложенной выборке с лучшими параметрами
best_model_rf = grid_search_rf.best_estimator_
test_pred_rf = best_model_rf.predict(test_data)
test_acc_rf = accuracy_score(test_labels, test_pred_rf)
print("Test accuracy for RandomForestClassifier:", test_acc_rf)

best_model_svm = grid_search_svm.best_estimator_
test_pred_svm = best_model_svm.predict(test_data)
test_acc_svm = accuracy_score(test_labels, test_pred_svm)
print("Test accuracy for SVM:", test_acc_svm)

In [ ]:
pd.set_option('display.max_colwidth', None)
results = pd.DataFrame({
    'model': ['Random Forest', 'SVM'],
    'best parameters': [grid_search_rf.best_params_, grid_search_svm.best_params_],
    'best score': [grid_search_rf.best_score_, grid_search_svm.best_score_],
    'test accuracy': [test_acc_rf, test_acc_svm]
})
print(results)